# 1. Brain Source Localization with ConvDip

In [1]:
%run functions.ipynb

In [2]:
"""
The subject underwent four tasks, and corresponding EEG data were collected.
task list: ['LA','LV','RA','RV']
"""
# you can choose single or multiple task ids from task list ['LA','LV','RA','RV']
tasks = ['LA', 'LV'] # or 'LA' or ['LA'], etc.
# set your result path
result_path = './result/'
ConvDip_ESI(tasks, result_path)

processing task: LA
(59, 3)
result saved in: ./result//sample/Test_result_evoked_LA.mat
processing task: LV
(59, 3)
result saved in: ./result//sample/Test_result_evoked_LV.mat


In [3]:
# choose only ONE task from ['LA','LV','RA','RV']
task = 'LA' 
s_pred = load_result(task, result_path)
print(s_pred.shape) # s_pred: estimated sources at different timepoints

load result for task: LA
(1984, 241)


# 2. 3D Visualization

In [4]:
%run brain.ipynb

In [5]:
"""
hemi: Hemisphere id (ie ‘lh’, ‘rh’, ‘both’, or ‘split’). 
In the case of ‘lh’, only left hemisphere is shown in the window. 
In the case of ‘rh’, only right hemispheres is shown in the window. 
In the case of ‘both’, both hemispheres are shown in the same window. 
In the case of ‘split’ hemispheres are displayed side-by-side in different viewing panes.
"""

hemi='split' # choose from ['lh', 'rh', 'split', 'both']
brain3d(s_pred, hemi)


Opening raw data file C:\Users\dimpa\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...


    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
319 events found on stim channel STI 014
Event IDs: [ 1  2  3  4  5 32]
Removing projector <Projection | Average EEG reference, active : False, n_channels : 60>
Not setting metadata
72 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] s
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
Loading data for 72 events and 106 original time points ...
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1711']
    Rejecting  epoch based on EOG : ['EOG 061']
  

In [6]:
import mne
from mne import read_surface
vertices, faces = mne.read_surface('C:/Users/dimpa/mne_data/MNE-sample-data/subjects/sample/surf/lh.pial')
print(vertices)
print(faces)
print(s_pred)

[[-21.12763786 -88.4329071   13.29888344]
 [-21.47982788 -88.38152313  13.2219944 ]
 [-21.88400841 -88.28314209  13.2074194 ]
 ...
 [-13.28218079  75.84895325  57.0162735 ]
 [-15.01982021  75.05326843  58.20228577]
 [-16.47038078  74.3368454   56.61045456]]
[[     0      1      3]
 [     4      3      1]
 [     0     61     62]
 ...
 [153723 153732 153919]
 [153900 153907 153901]
 [154073 154322 154245]]
[[0.00969072 0.00967974 0.01275246 ... 0.01222476 0.01176346 0.01120899]
 [0.02300234 0.02311075 0.02096353 ... 0.02240465 0.02239515 0.02228532]
 [0.00612901 0.00726201 0.00051712 ... 0.002742   0.00300734 0.00319636]
 ...
 [0.00467773 0.00444413 0.00331879 ... 0.00406433 0.0043078  0.00445012]
 [0.00331646 0.00318075 0.00244875 ... 0.00299053 0.00316049 0.00339406]
 [0.00530668 0.00593612 0.00373083 ... 0.00359215 0.00380004 0.00402161]]


In [7]:
vertices_list = vertices.tolist()  # Convert vertices to list
faces_list = faces.tolist()  # Convert faces to list
s_pred_list = s_pred.tolist()  # Convert s_pred matrix to list


data = {
    "vertices": vertices_list,
    "faces": faces_list,
    "activity": s_pred_list
}

In [10]:
import json
# Serialize data to JSON string
json_str = json.dumps(data, indent=4)

# To save this JSON string to a file
with open('brain_data.json', 'w') as json_file:
    json_file.write(json_str)

In [ ]:
import mne
from pathlib import Path
import scipy.io

# Define the paths to the folders
data_path = Path("C:/Users/dimpa/mne_data/MNE-sample-data/")
subjects_dir = data_path / "subjects"

# Specify the subject and the surface to use
subject = "sample"
surf = "inner_skull"  # Use the inner surface of the skull

# Load the brain surface mesh
vertices, faces = mne.read_surface(str(subjects_dir / subject / 'bem' / f'{surf}.surf'))

# Load detected cells coordinates from .mat file
mat_data = scipy.io.loadmat('C:/Users/dimpa/OneDrive - stevens.edu/SSW-555/Existing Solution/ESI_project 2/ESI_project 2/result/sample/Test_result_evoked_LA.mat')
detected_cells_coordinates = mat_data['s_pred']

# Save the vertices and faces to an .obj file
output_obj_path = "reddetected_cells.obj"
with open(output_obj_path, 'w') as obj_file:
    # Write vertices for brain surface
    for vertex in vertices:
        obj_file.write(f'v {vertex[0]} {vertex[1]} {vertex[2]} 0.5 0.5 0.5\n')  # Gray color for brain surface
    
    # Write vertices for detected cells with red color
    for cell_coordinates in detected_cells_coordinates:
        obj_file.write(f'v {cell_coordinates[0]} {cell_coordinates[1]} {cell_coordinates[2]} 1.0 0.0 0.0\n')  # Red color for detected cells

    # Write faces
    for face in faces:
        obj_file.write(f'f {" ".join(str(idx + 1) for idx in face)}\n')

print(f"Obj file saved: {output_obj_path}")
